In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras

In [ ]:
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import getpass

In [ ]:
if openai.api_key is None:
  openai.api_key = getpass.getpass(prompt='Please enter your OpenAI API key: ')

Please enter your OpenAI API key: ··········


# Document Loading and Preparation

In [ ]:
data_url ="https://archive.org/stream/j-r-r-tolkien-lord-of-the-rings-01-the-fellowship-of-the-ring-retail-pdf/j-r-r-tolkien-lord-of-the-rings-01-the-fellowship-of-the-ring-retail-pdf_djvu.txt"

In [ ]:
filepath = keras.utils.get_file('lotr.txt',data_url)

   8192/Unknown - 0s 0us/step

In [ ]:
with open(filepath) as f:
  raw_text=f.read()

In [ ]:
raw_text

'<!DOCTYPE html>\n<html lang="en">\n<!-- __ _ _ _ __| |_ (_)__ _____\n    / _` | \'_/ _| \' \\| |\\ V / -_)\n    \\__,_|_| \\__|_||_|_| \\_/\\___| -->\n  <head data-release=ab26fe23 data-node="www14.us.archive.org">\n    <title>Full text of &quot;j-r-r-tolkien-lord-of-the-rings-01-the-fellowship-of-the-ring-retail-pdf&quot;</title>\n\n          <meta name="viewport" content="width=device-width, initial-scale=1.0"/>\n    \n        <meta name="google-site-verification" content="Q2YSouphkkgHkFNP7FgAkc4TmBs1Gmag3uGNndb53B8" />\n    <meta name="google-site-verification" content="bpjKvUvsX0lxfmjg19TLblckWkDpnptZEYsBntApxUk" />\n\n    <script >\n/* @licstart  The following is the entire license notice for the\n * JavaScript code in this page.\n *\n * This program is free software: you can redistribute it and/or modify\n * it under the terms of the GNU Affero General Public License as published by\n * the Free Software Foundation, either version 3 of the License, or\n * (at your option) any la

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 100,
    length_function=len,

)

In [ ]:
chunks = text_splitter.split_text(raw_text)

In [ ]:
len(chunks)

1130

## Pandas Dataframe

In [ ]:
pre_upsert_df2 = pd.DataFrame(columns=['id','values','metadata'])

In [ ]:
i=0
id_no=200
for chunk in chunks:
  pre_upsert_df.loc[i]= [id_no, get_embedding(chunk, engine="text-embedding-ada-002"),{'text':chunk}]
  i+=1
  id_no+=10001

In [ ]:
pre_upsert_df

,id,values,metadata
0,200,"[-0.010922838933765888, 0.003157747909426689, ...","{'text': '<!DOCTYPE html> <html lang=""en""> <!-..."
1,10201,"[-0.016237545758485794, 0.011505577713251114, ...",{'text': '<script > /* @licstart The followin...
2,20202,"[-0.018369121477007866, 0.01740444079041481, -...","{'text': '<script src=""//archive.org/includes/..."
3,30203,"[-0.0018431070493534207, 0.01481369324028492, ...",{'text': '<!-- Wraps all page content --> ...
4,40204,"[-0.02332243323326111, 0.002449121791869402, 0...",{'text': '.login-button-1 .dropdown-toggle.log...
...,...,...,...
1125,11251325,"[-0.012183423154056072, -0.009419488720595837,...",{'text': '</pre> </div><!--/.container-...
1126,11261326,"[-0.015515564009547234, 0.0027221895288676023,...",{'text': '|AdapterPipeline.inc:757:execute_pip...
1127,11271327,"[-0.006065084598958492, 0.0011946912854909897,...",{'text': '|download.php:926:head ...
1128,11281328,"[-0.02487282268702984, 0.00659136800095439, -0...",{'text': 'archive_analytics.process_url_events...


In [ ]:
pre_upsert_df.to_csv('/content/drive/MyDrive/Ragtaime/ring_ring.csv')